In [1]:
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Alias _set_env to set_env
set_env = _set_env

set_env("OPENAI_API_KEY")

In [2]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "diary"

In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [12]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///diary.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['diary_entries']


In [13]:
from sqlalchemy import create_engine  # Import create_engine
from sqlalchemy.orm import sessionmaker
from create_diary_schema import DiaryEntry

# Connect to the SQLite database
engine = create_engine("sqlite:///diary.db")
Session = sessionmaker(bind=engine)
session = Session()

# Query all diary entries
entries = session.query(DiaryEntry).all()
for entry in entries:
    print(f"Date: {entry.date}, Content: {entry.content[:50]}...")  # Print the first 50 characters

Date: 2023-03-28, Content: - i would like to spend a day more in my body and ...
Date: 2025-04-08, Content: Today is 8th of April 2025. I want to honor this d...
Date: 2023-09-12, Content: - value of the day. compassion.
- this morning i w...
Date: 2024-03-12, Content: - encuentro con el monje induista de bilbao.
- dic...
Date: 2024-01-17, Content: - en la meditacion. tension entre el hyperachiever...
Date: 2024-09-09, Content: - first day of coworking at commonshub. i met a gi...
Date: 2024-03-06, Content: - learning to see everything with curiosity
- me l...
Date: 2025-04-04, Content: I am somewhere on the 4th of April 2025 and it's l...
Date: 2025-03-28, Content: - Today is 28th of March 2025. It's in the morning...
Date: 2022-12-24, Content: - la pregunta inicial. what is the spiritualist mo...
Date: 2023-08-25, Content: - el tiempo invertido en cuidados personales. cuid...
Date: 2024-06-20, Content: - i know it's difficult to change our daily habits...
Date: 2025-03-08, Content: H

In [14]:


from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [15]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [16]:
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 2
for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}
================================ Human Message =================================

Question: {input}


In [17]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [10]:
write_query({"question": "when was the first entry?"})

{'query': 'SELECT date FROM diary_entries ORDER BY date ASC LIMIT 1;'}

In [18]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [19]:
execute_query({"query": "SELECT COUNT(*) AS total_entries FROM diary_entries;"})

{'result': '[(115,)]'}

In [20]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

In [21]:
from telegram.client import Telegram

tg = Telegram(
    api_id=26621694,
    api_hash='d5ca85e3ee9c5232b3a7afc537c59452',
    phone='+32484834767',
    database_encryption_key='BRU0109al',
)

[ 3][t 0][1747215872.447763681][Client.cpp:600]	Create client 1
[ 3][t 0][1747215872.448188304][Client.cpp:383]	Initialize client 1


In [ ]:
tg.login()

[ 2][t 1][1747215877.240357398][TdDb.cpp:431][#1][!RunOnSchedulerWorker]	Set PRAGMA user_version = 14
[ 2][t 4][1747215877.280628681][AuthDataShared.cpp:118][#1][!Td]	DcId{1} [auth_key_id:0][state:Empty][created_at:0][last_used:0]
[ 2][t 4][1747215877.288210868][Session.cpp:271][#1][!SessionProxy:1:main]	Generate new session_id 13273179412260031627 for auth key 0 for main DC1
[ 2][t 4][1747215879.461263656][AuthDataShared.cpp:118][#1][!Session:1:main]	DcId{1} [auth_key_id:10336560890820822314][state:NoAuth][created_at:1747215877][last_used:0]
[ 2][t 4][1747215879.461313962][Session.cpp:1422][#1][!Session:1:main]	Update auth key in session_id 13273179412260031627 to 10336560890820822314
[ 2][t 4][1747215879.682384729][AuthDataShared.cpp:118][#1][!Session:1:main]	DcId{4} [auth_key_id:0][state:Empty][created_at:0][last_used:0]
[ 2][t 4][1747215879.684224367][Session.cpp:271][#1][!SessionProxy:4:main]	Generate new session_id 10302426312079727728 for auth key 0 for main DC4
[ 2][t 4][174721

<AuthorizationState.READY: 'authorizationStateReady'>

[ 2][t 4][1747215899.194118022][Session.cpp:271][#1][!SessionProxy:1:main]	Generate new session_id 2099909725670811670 for auth key 10336560890820822314 for DC1
[ 2][t 4][1747215899.453187227][AuthDataShared.cpp:118][#1][!Session:1:main]	DcId{1} [auth_key_id:10336560890820822314][state:OK][created_at:1747215877][last_used:1747332878]


In [23]:
result = tg.get_chats()
result.wait()
print(result.update)


{'@type': 'chats', 'total_count': 93, 'chat_ids': [777000, -4762585149, 1771869684, -1001619831579, 6822025926, 7816829551, 7846382525, 57098707, -1002340503783, 7954296876, 1241442881, 7069489447, 8078951456, 7801784533, -857213611, 7595113840, -1001895831685, 735511181, 7712627342, 7120032145, 7870468219, 6597325114, 7591611458, 7672348965, -4017208805, 7662181205, -4150664340, 7945116301, 1144008385, 6163574533, 7878237007, 6758663599, 7129916042, 7525131981, 7401048146, 7127173154, 7152100632, 7019764895, 7298823424, 7247728024, 6381639412, 7076346379, 7176466021, 7028967332, 7192362684, 7084784023, 7106241006, 1318953231, 7150614755, 889765482, 7197974963, 7056847755, 6441630674, 6952861107, 1530754544, 6856322095, 6633835366, 6976665394, 6686797836, 6709461690, 6533868226, 6943892037, 6772438286, -4014980841, -4081098759, 6678459433, 6638828584, 6505532685, 6414335330, 6764866171, 6454772031, 6928544385, 6663733510, 6531836133, 6465153473, 6631804287, 6622357982, 6097187659, 1097

In [24]:
chat_id = -4762585149

tg.send_message(
            chat_id=chat_id,
            text='test',
        )


In [25]:
def post_to_telegram(state: State):
    """Post the generated answer to Telegram."""
    chat_id = -4762585149 # Replace with the desired chat ID
    message = state["answer"]  # Use the answer from the state
    try:
        tg.send_message(chat_id=chat_id, text=message)
        return {"answer": state["answer"], "status": "Message sent"}  # Update the state
    except Exception as e:
        return {"answer": state["answer"], "status": f"Failed to send message: {str(e)}"}  # Update the state

In [26]:
from langgraph.graph import START, StateGraph

workflow = StateGraph(State)
workflow.add_node("write_query", write_query)
workflow.add_node("execute_query", execute_query)
workflow.add_node("generate_answer", generate_answer)
workflow.add_node("post_to_telegram", post_to_telegram)  # Add the new node

workflow.add_edge(START, "write_query")
workflow.add_edge("write_query", "execute_query")
workflow.add_edge("execute_query", "generate_answer")
workflow.add_edge("generate_answer", "post_to_telegram")  # Connect the new node

memory = MemorySaver()

graph = workflow.compile(interrupt_before=["post_to_telegram"], checkpointer=memory)


In [27]:
# Configuration
config = {"configurable": {"thread_id": "3"}}

initial_input = {"question": HumanMessage(content="hi i am manuel, the owner of the diary")}

for chunk in graph.stream(initial_input, config, stream_mode="updates"):
    print("Chunk Metadata Overview:")
    for key, value in chunk.items():
        print(f"{key}: {type(value)}")
    print("Detailed Chunk Content:")
    print(chunk)

# Stream updates
#for chunk in graph.stream(initial_input, config, stream_mode="values"):
#    chunk["answer"].pretty_print()
#    print(chunk)



Chunk Metadata Overview:
write_query: <class 'dict'>
Detailed Chunk Content:
{'write_query': {'query': "SELECT id, date, content FROM diary_entries WHERE content LIKE '%hi i am manuel, the owner of the diary%' LIMIT 10;"}}
Chunk Metadata Overview:
execute_query: <class 'dict'>
Detailed Chunk Content:
{'execute_query': {'result': ''}}
Chunk Metadata Overview:
generate_answer: <class 'dict'>
Detailed Chunk Content:
{'generate_answer': {'answer': 'Based on the SQL query and the SQL result, it appears that the system is searching for diary entries that contain the specific content "hi i am manuel, the owner of the diary". However, since no SQL result is provided, it can be inferred that there are no entries in the database that match this content.\n\nTherefore, the answer to the user question would be:\n\n**There are no diary entries that match the content "hi i am manuel, the owner of the diary".**'}}
Chunk Metadata Overview:
__interrupt__: <class 'tuple'>
Detailed Chunk Content:
{'__inte

In [30]:
# Configuration
config = {"configurable": {"thread_id": "33"}}

initial_input = {
    "question": HumanMessage(
        content=(
            "you are a diary drafting assistant with access to my diary entries. "
            ""
            "i would like you to send life updates to my parents in spanish."
            "draft a short message summarizing what is going on in my life based on latest diary entries. "
            "please only use the content of the entries to generate the answer. "
            "do not include any system message nor AI message in the generated answer, just the content of the entries."
            "do not include capital letters in the message. "
            "do make 1 mistake in spelling in the message. "
            "please speak as if you were me and you are speaking directly to my parents. "
            "please use coloquial spanish from spain. avoid using latin american spanish. "
            "please do not mention any dates in the message. just speak as if is today"
            "do not include introduction sentence. this is going to be a telegram message."
        )
    )
}

# Stream through the graph until the interrupt
for step in graph.stream(initial_input, config, stream_mode="updates"):
    if "generate_answer" in step:  # Check if the generate_answer node is reached
        print("Generated Answer:")
        print(step["generate_answer"]["answer"])  # Display the answer to the user
    print(step)

# Ask for user approval to send the message
user_approval = input("Do you want to send this answer to Telegram? (yes/no): ")

# Check user approval
if user_approval.lower() == "yes":
    # Continue the graph execution to send the message
    for step in graph.stream(None, config, stream_mode="values"):
        if "post_to_telegram" in step:  # Check if the post_to_telegram node is reached
            print("Message sent to Telegram.")
else:
    print("Operation cancelled by user.")



{'write_query': {'query': 'SELECT content FROM diary_entries ORDER BY date DESC LIMIT 10'}}
{'execute_query': {'result': '[(\'In the meditation this morning, I become aware of the extreme fragility of my attention, of my freedom, of my peace. I try to focus on the here and now, and my mind goes constantly without me, allowing it towards productivity. I need to learn to stay less on my mind, on my own abstractions, and...\',), (\'Every day. The song of the clock, of the bell of the Church of Bridgetines, which is close to high place, reminds me of my committed to ground. Stop traveling with my heads and go into the concrete. Connect with real people with real problems, try to solve them. Stop dreaming and the biggest...\',), ("Today is 10th of May. Good morning. Dear Diary, I had a good night\'s sleep and I feel refreshed. I\'m in the terrace. It\'s almost 8 o\' clock in the morning. I hear a sound already from works outside. Somebody must be cutting some trees or some leaves or cleanin